In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langfuse import observe
from langfuse.langchain import CallbackHandler
from langchain.tools import tool
from langchain.agents import create_agent
from dotenv import load_dotenv

load_success = load_dotenv()

In [ ]:
# Langchain with simple example
# Then we will try types of prompting
    # 1. Zero Shot Prompt
    # 2. One Shot 
    # 3. Few Shot Prompt 
    # 4. Chain of Though
# RAG with PDF extraction

# Simple translation pipeline using LangChain and OpenAI

# Language
# Input Text

# Transalated Text

# 1. Create LLM Object
# 2. Build a Prompt Template
# 3. Compose the prompt and model
# 4. Execute the chain

# 1. Create an LLM client object and we are going to use gpt 4 model
#    - 'ChatOpenAI' is a LangChain wrapper around a chat model.
#    - model="gpt-4o-mini" selects which model to talk to.
#    - You can later pass settings like temperature, max_tokens, streaming, etc.
llm = ChatOpenAI(model="gpt-4o-mini")

# 2. Prompting is a we can communicate with LLM
# 2. Build a PromptTemplate.
#    - input_variables defines placeholders we will fill later.
#    - template is the string with placeholders: {language} and {text}.
#    - The template ensures we send a consistent instruction to the model.

prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="Translate the following sentence into {language}:\n\n{text}",
)

# 3. Compose the prompt and model into a chain using the pipe operator.
#    - This expression wires the template output into the LLM as input.
#    - In other words: when executed, the filled-in prompt is sent to `llm`.
#    - Many LangChain versions allow "piping" components like this for a simple flow.
chain = prompt | llm

# 4. Execute the chain.
#    - invoke takes a dict mapping input variable names to values.
#    - It runs the prompt template (fills placeholders), sends it to the LLM and returns a response object.

text = "Good morning, Whats going on??"
language = "French"

result = chain.invoke({"text": text, "language": language})

print("--- Translated Text ---")
print(result.content)


# The model performs a task without seeing any examples.
# You just give an instruction — the model uses its internal knowledge to answer.

# Example
# Classify the sentiment of the following text as 'positive' or 'negative'.
# Do not generate any additional texts or explainations, return only the sentiment.
# Your output should be one of the following:["positive",negative"]
# Text: {query}
# Sentiment:

# ----------------------------------------------------------------------
# One-Shot Prompting
# You provide one example of how the task should be done before asking for the actual output.

# Example

# Classify the sentiment of the following text as 'positive' or 'negative'.
# Do not generate any additional texts or explainations, return only the sentiment.
# Your output should be one of the following:["positive",negative"]
# Example 01
# Text: The film was good but hate the villain, I hate vaccations, The film was good, but i dont like the climax, 
# Sentiment: Positive

# Text:{query} Sentiment:

--- Translated Text ---
Bonjour, qu'est-ce qui se passe ?


In [ ]:

# The model performs a task without seeing any examples.
# You just give an instruction — the model uses its internal knowledge to answer.

# Example
# Classify the sentiment of the following text as 'positive' or 'negative'.
# Do not generate any additional texts or explainations, return only the sentiment.
# Your output should be one of the following:["positive",negative"]
# Text: {query}
# Sentiment:

# ----------------------------------------------------------------------
# One-Shot Prompting
# You provide one example of how the task should be done before asking for the actual output.

# Example

# Classify the sentiment of the following text as 'positive' or 'negative'.
# Do not generate any additional texts or explainations, return only the sentiment.
# Your output should be one of the following:["positive",negative"]
# Example 01
# Text: The film was good but hate the villain, I hate vaccations, The film was good, but i dont like the climax, 
# Sentiment: Positive

# Text:{query} Sentiment:

# ----------------------------------------------------------------------
# Few-Shot Prompting
# You provide one example of how the task should be done before asking for the actual output.

# Example

# Classify the sentiment of the following text as 'positive' or 'negative'.
# Do not generate any additional texts or explainations, return only the sentiment.
# Your output should be one of the following:["positive",negative"]
# Example 01
# Text: The film was good but hate the villain # I hate vaccations, The film was good, but i dont like the climax, 
# Sentiment: Positive
# Example 02
# Text: I hate vaccations
# Sentiment: Positive

# Text:{query} Sentiment:

# ----------------------------------------------------------------------
# Chain of Thought Prompting
# You ask the model to “think step-by-step” before answering. Helps with reasoning, math, and logical tasks.

# Example
# If a train leaves at 3PM and arrives at 6PM, how long was the journey?
# Let's think step by step.

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

input_prompt="""
If a train leaves at 3PM and arrives at 6PM, how long was the journey?
Let's think step by step.
"""
query = ""

prompt = PromptTemplate(
    input_variables=["query"],
    template=input_prompt,
)
chain = prompt | llm

result = chain.invoke({"query": query})

print("--- Translated Text ---")
print(result.content)

--- Translated Text ---
To determine the duration of the train journey, we can follow these steps:

1. **Identify the departure time**: The train leaves at 3 PM.
2. **Identify the arrival time**: The train arrives at 6 PM.
3. **Calculate the duration**: To find the duration, we subtract the departure time from the arrival time.

   - From 3 PM to 4 PM is 1 hour.
   - From 4 PM to 5 PM is another hour (totaling 2 hours).
   - From 5 PM to 6 PM is yet another hour (totaling 3 hours).

So, the total journey time is **3 hours**.


In [ ]:
# 1. Initialize LangFuse Client
# 2. Define Observable Function
# 3. Initialize LLM
# 4. Create Prompt Template
# 5. Create and Run Chain
# 6. Return output content

# ---------------------------------------------
# 🔹 STEP 1: Initialize LangFuse Client
# ---------------------------------------------
langfuse_handler = CallbackHandler()  # Creates callback for tracing


# ---------------------------------------------
# 🔹 STEP 2: Define Observable Function
# ---------------------------------------------
@observe(name="motivational_quote_flow")
def get_quote(language: str):
    """Generate a motivational quote in the given language."""

    # 🔹 STEP 3: Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", callbacks=[langfuse_handler])

    # 🔹 STEP 4: Create Prompt Template
    prompt = PromptTemplate(
        input_variables=["language"],
        template="Write a short motivational quote in {language}.",
    )

    # 🔹 STEP 5: Create and Run Chain
    chain = prompt | llm
    result = chain.invoke({"language": language})

    # 🔹 STEP 6: Return output content
    return result.content


# 🔹 STEP 7: Test Function
print(get_quote("English"))


"Believe in the power of your dreams; they are the blueprints of your future."


In [4]:
from pydantic import BaseModel, Field

# ---------------------------------------------
# 🔹 Step 1: Define Output Schema
# ---------------------------------------------
class ResponseSchema(BaseModel):
    """Contact information for a person."""

    calculation: str = Field(description="The mathematical expression")
    result: str = Field(description="The numerical answer")
    explanation: str = Field(description="Step-by-step explanation")


# ---------------------------------------------
# 🔹 Step 2: Define a Tool
# ---------------------------------------------
@tool
def calculator(expression: str) -> str:
    """Evaluates a math expression and returns the result."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"


# ---------------------------------------------
# 🔹 Step 3: Initialize LLM and Agent
# ---------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
agent = create_agent(
    model=llm,
    tools=[calculator],
    debug=False,
    response_format=ResponseSchema,
    system_prompt="You are a Mathematical assistant.",
)

# ---------------------------------------------
# 🔹 Step 4: Ask a question
# ---------------------------------------------
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What is 25 * 12 + 36?"}]}
)

print(result["structured_response"])

calculation='25 * 12 + 36' result='336' explanation='First, multiply 25 by 12, which equals 300. Then, add 36 to 300, resulting in 336.'


In [5]:
# Indexing
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings


# 1. Load PDF
pdf_path = "./pdf/pondicherry.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# 2. Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(docs)

# 3. Create embeddings & store in Chroma
embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")

In [6]:
# Retrieval and Generation
question = "What are the main attractions in Pondicherry?"

# 1. Load Chroma DB
embeddings = OpenAIEmbeddings()
vector_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

# 2. Retrieve relevant chunks
retrieved_docs = vector_db.similarity_search(question ,k=2)

# 3. Combine retrieved text into context
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 4. Create the system + user message
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# 5. Create ChatOpenAI and call it
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 6. Build the message sequence manually
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
]

response = llm.invoke(messages)

print("-" * 50)
print(f"Question: {question}")
print("-" * 50)
print(f"Answer: {response.content}")
print("-" * 50)
print(f"Source documents used: {len(response.content)} chunks")


--------------------------------------------------
Question: What are the main attractions in Pondicherry?
--------------------------------------------------
Answer: The main attractions in Pondicherry include the Sri Aurobindo Ashram, various old churches from the 18th and 19th centuries, and heritage buildings around Promenade Beach such as the Gandhi statue, French War Memorial, and the 19th Century Lighthouse. Other notable sites are the Puducherry Botanical Gardens, Chunnambar Backwater resort, and the Arulmigu Manakula Vinayagar Devasthanam temple. Additionally, visitors can explore the Romain Rolland Library, Pondicherry Museum, and the French Institute of Pondicherry.
--------------------------------------------------
Source documents used: 518 chunks
